# Data Cleaning

1) Concatenating csv files

2) Remove Duplicates

3) BookingIDs are present in features csv file, but not in the labels csv file. 

4) Saving clean Dataframes into csv files

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
import matplotlib.pyplot as plt
from pprint import pprint
sb.set()
from pathlib import Path
import timeit

In [3]:
def unique(a):
    """ return the list with duplicate elements removed """
    return list(set(a))

def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

def not_intersect(a, b):
    """ return the intersection of two lists """
    return list(set(union(a, b)) - set(intersect(a,b)))

def find_dupes(ID_list):
    seen = {}
    duplicates = []

    for x in ID_list:
        if x not in seen:
            seen[x] = 1
        else:
            if seen[x] == 1:
                duplicates.append(x)
            seen[x] += 1
    return duplicates

In [4]:
df0 = pd.read_csv('safety/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df1 = pd.read_csv('safety/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df2 = pd.read_csv('safety/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df3 = pd.read_csv('safety/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df4 = pd.read_csv('safety/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df5 = pd.read_csv('safety/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df6 = pd.read_csv('safety/features/part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df7 = pd.read_csv('safety/features/part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df8 = pd.read_csv('safety/features/part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
df9 = pd.read_csv('safety/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')

df_labels = pd.read_csv('safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv')

In [5]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]
df = pd.concat(frames, ignore_index=True).sort_values(['bookingID', 'second']).reset_index(drop=True)
# Remove speed<0 and second>10000
df = df[df['Speed']>=0]
df = df[df['second']<=10000]
print(df.shape)
df.head()

(15895173, 11)


,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991
1,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,0.228454
2,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454
3,0,8.0,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454
4,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,0.228454


## Remove Duplicates

In [6]:
df_labels = pd.read_csv('safety/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv')
df_labels = df_labels.sort_values('bookingID').reset_index(drop=True)

bookingID_list = df['bookingID'].unique().tolist()
bookingID_list_labels = df_labels['bookingID'].tolist()

df_dupes = find_dupes(bookingID_list)
df_labels_dupes = find_dupes(bookingID_list_labels)

print(df_dupes, '\n')
print(df_labels_dupes)

[] 

[13, 154618822837, 223338299461, 395136991308, 403726925929, 455266533495, 481036337234, 515396075694, 695784702084, 919123001348, 970662608932, 1279900254294, 1348619731077, 1391569403991, 1408749273124, 1511828488211, 1632087572573, 1649267441751]


In [7]:
for dupes in df_labels_dupes:
    temp_df = df_labels.loc[df_labels['bookingID']==dupes]['label']
    same = temp_df.iloc[0]
    print('Duplicate BookingID found :\t' + str(dupes))
    print(temp_df)
    for idx in range(0, len(temp_df.index)):
        if temp_df.iloc[idx]!=same:
            print('Not same')
            for idx2 in temp_df.index:
                df_labels.drop(idx2, axis=0, inplace=True) # Remove duplicates that have different values from df_labels
            print('Removed data for duplicate booking ID ' + str(dupes) + '\n\n')
            
df_labels = df_labels.sort_values('bookingID').reset_index(drop=True)

Duplicate BookingID found :	13
9     1
10    0
Name: label, dtype: int64
Not same
Removed data for duplicate booking ID 13


Duplicate BookingID found :	154618822837
2124    0
2125    1
Name: label, dtype: int64
Not same
Removed data for duplicate booking ID 154618822837


Duplicate BookingID found :	223338299461
3031    1
3032    0
Name: label, dtype: int64
Not same
Removed data for duplicate booking ID 223338299461


Duplicate BookingID found :	395136991308
5124    0
5125    1
Name: label, dtype: int64
Not same
Removed data for duplicate booking ID 395136991308


Duplicate BookingID found :	403726925929
5242    0
5243    1
Name: label, dtype: int64
Not same
Removed data for duplicate booking ID 403726925929


Duplicate BookingID found :	455266533495
5953    0
5954    1
Name: label, dtype: int64
Not same
Removed data for duplicate booking ID 455266533495


Duplicate BookingID found :	481036337234
6288    1
6289    0
Name: label, dtype: int64
Not same
Removed data for duplicate booking

In [8]:
print(df_labels.shape)
df_labels.head()

(19982, 2)


,bookingID,label
0,0,0
1,1,1
2,2,1
3,4,1
4,6,0


## Remove Datapoints that are in df but are not present in df_labels and vice versa

In [9]:
bookingID_list = df['bookingID'].unique().tolist()
bookingID_list_labels = df_labels['bookingID'].tolist()
ids_to_remove = not_intersect(bookingID_list, bookingID_list_labels)
print(not_intersect(bookingID_list, bookingID_list_labels))
len(not_intersect(bookingID_list, bookingID_list_labels))

[695784702084, 1348619731077, 438086664198, 541165879428, 919123001348, 970662608908, 13, 146028888209, 446676598802, 1511828488211, 1391569404051, 1279900254359, 979252543641, 292057776283, 515396075679, 1408749273124, 970662608932, 1382979469484, 1039382085805, 335007449134, 515396075694, 1494648619181, 678604832817, 1434519077043, 154618822837, 1666447310903, 223338299461, 936302870600, 326417514568, 395136991308, 1408749273165, 481036337234, 82, 1279900254294, 1391569403991, 1090921693272, 1649267441751, 1176821039197, 1632087572573, 1236950581343, 223338299491, 403726925929, 833223655531, 455266533495, 223338299515]


45

In [10]:
indexes_to_remove_for_df = []
for i in ids_to_remove:
    idx_to_remove = df.loc[df['bookingID']==i].index.tolist()
    for j in idx_to_remove:
        indexes_to_remove_for_df.append(j)

print(indexes_to_remove_for_df)
len(indexes_to_remove_for_df)

[7069872, 7069873, 7069874, 7069875, 7069876, 7069877, 7069878, 7069879, 7069880, 7069881, 7069882, 7069883, 7069884, 7069885, 7069886, 7069887, 7069888, 7069889, 7069890, 7069891, 7069892, 7069893, 7069894, 7069895, 7069896, 7069897, 7069898, 7069899, 7069900, 7069901, 7069902, 7069903, 7069904, 7069905, 7069906, 7070067, 7070068, 7070069, 7070070, 7070071, 7070072, 7070073, 7070074, 7070075, 7070076, 7070077, 7070078, 7070079, 7070080, 7070081, 7070082, 7070083, 7070084, 7070085, 7070086, 7070087, 7070088, 7070089, 7070090, 7070091, 7070092, 7070093, 7070094, 7070095, 7070096, 7070097, 7070098, 7070099, 7070100, 7070101, 7070102, 7070103, 7070104, 7070105, 7070106, 7070107, 7070108, 7070109, 7070110, 7070111, 7070112, 7070113, 7070114, 7070115, 7070116, 7070117, 7070118, 7070119, 7070120, 7070121, 7070122, 7070123, 7070124, 7070125, 7070126, 7070127, 7070128, 7070129, 7070130, 7070131, 7070132, 7070133, 7070134, 7070135, 7070136, 7070137, 7070138, 7070139, 7070140, 7070141, 7070142, 

18523

In [11]:
# Remove "not_intersect(bookingID_list, bookingID_list_labels)" indexes from df
df = df.drop(indexes_to_remove_for_df, axis=0).sort_values(['bookingID', 'second']).reset_index(drop=True)
print(df.shape)

(15876650, 11)


In [12]:
indexes_to_remove_for_df_labels = []
for i in ids_to_remove:
    idx_to_remove = df_labels.loc[df_labels['bookingID']==i].index.tolist()
    for j in idx_to_remove:
        indexes_to_remove_for_df_labels.append(j)
    
print(indexes_to_remove_for_df_labels) 
len(indexes_to_remove_for_df_labels)

[5627, 6904, 11973, 1958, 5771, 16633, 15187, 12159, 3826, 6777, 16534, 12853, 4308, 17825, 8575, 17086, 19535, 11647, 4197, 16723, 50, 13269, 14363, 14962, 3044, 10416, 3063]


27

In [13]:
# Remove "not_intersect(bookingID_list, bookingID_list_labels)" indexes from df_labels
df_labels = df_labels.drop(indexes_to_remove_for_df_labels, axis=0).sort_values('bookingID').reset_index(drop=True)
print(df_labels.shape)

(19955, 2)


## Final number of BookingIDs

In [14]:
# Checking that both have the same unique IDs
bookingID_list_new = df['bookingID'].unique().tolist()
bookingID_list_labels_new = df_labels['bookingID'].unique().tolist()

print(len(bookingID_list_new))
print(len(bookingID_list_labels_new))
print(bookingID_list_new == bookingID_list_labels_new)

19955
19955
True


## Save cleaned Dataframes

In [15]:
print('Start')

df_labels.to_csv(r'safety/data_cleaned_labels.csv', index=False)
print("'data_cleaned_labels.csv' Done")

df.to_csv(r'safety/data_cleaned_features.csv', index=False)
print("'data_cleaned.csv' Done")

print('Finished')

Start
'data_cleaned_labels.csv' Done
'data_cleaned.csv' Done
Finished


## Get combined DataFrame

In [16]:
no_of_chunks = 1000

no_of_indexes = df.shape[0]
divided_chunk = int(no_of_indexes/no_of_chunks)

print(divided_chunk) # Divide dataset into 4
print(no_of_indexes)

15876
15876650


In [17]:
df_chunks = [pd.DataFrame() for _ in range(no_of_chunks)]

for i in range(no_of_chunks-1):
    df_chunks[i] = df[divided_chunk*i:divided_chunk*(i+1)]
    
df_chunks[no_of_chunks-1] = df[divided_chunk*(no_of_chunks-1):no_of_indexes]

In [18]:
count=0
for i in df_chunks:
    print('Chunk ' + str(count) + ':')
    print(i.index)
    count+=1

Chunk 0:
RangeIndex(start=0, stop=15876, step=1)
Chunk 1:
RangeIndex(start=15876, stop=31752, step=1)
Chunk 2:
RangeIndex(start=31752, stop=47628, step=1)
Chunk 3:
RangeIndex(start=47628, stop=63504, step=1)
Chunk 4:
RangeIndex(start=63504, stop=79380, step=1)
Chunk 5:
RangeIndex(start=79380, stop=95256, step=1)
Chunk 6:
RangeIndex(start=95256, stop=111132, step=1)
Chunk 7:
RangeIndex(start=111132, stop=127008, step=1)
Chunk 8:
RangeIndex(start=127008, stop=142884, step=1)
Chunk 9:
RangeIndex(start=142884, stop=158760, step=1)
Chunk 10:
RangeIndex(start=158760, stop=174636, step=1)
Chunk 11:
RangeIndex(start=174636, stop=190512, step=1)
Chunk 12:
RangeIndex(start=190512, stop=206388, step=1)
Chunk 13:
RangeIndex(start=206388, stop=222264, step=1)
Chunk 14:
RangeIndex(start=222264, stop=238140, step=1)
Chunk 15:
RangeIndex(start=238140, stop=254016, step=1)
Chunk 16:
RangeIndex(start=254016, stop=269892, step=1)
Chunk 17:
RangeIndex(start=269892, stop=285768, step=1)
Chunk 18:
RangeInde

In [23]:
def match_ID_to_label(ID):
#     print(ID)
    return df_labels[df_labels['bookingID']==ID]['label'].iloc[0]

In [24]:
count=0
for chunk in df_chunks:
    chunk['label'] = chunk['bookingID'].apply(match_ID_to_label)
    print('Chunk ' + str(count) + ': ' + str(chunk.index) +  '\t -> DONE')
    count+=1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Chunk 0: RangeIndex(start=0, stop=15876, step=1)	 -> DONE
Chunk 1: RangeIndex(start=15876, stop=31752, step=1)	 -> DONE
Chunk 2: RangeIndex(start=31752, stop=47628, step=1)	 -> DONE
Chunk 3: RangeIndex(start=47628, stop=63504, step=1)	 -> DONE
Chunk 4: RangeIndex(start=63504, stop=79380, step=1)	 -> DONE
Chunk 5: RangeIndex(start=79380, stop=95256, step=1)	 -> DONE
Chunk 6: RangeIndex(start=95256, stop=111132, step=1)	 -> DONE
Chunk 7: RangeIndex(start=111132, stop=127008, step=1)	 -> DONE
Chunk 8: RangeIndex(start=127008, stop=142884, step=1)	 -> DONE
Chunk 9: RangeIndex(start=142884, stop=158760, step=1)	 -> DONE
Chunk 10: RangeIndex(start=158760, stop=174636, step=1)	 -> DONE
Chunk 11: RangeIndex(start=174636, stop=190512, step=1)	 -> DONE
Chunk 12: RangeIndex(start=190512, stop=206388, step=1)	 -> DONE
Chunk 13: RangeIndex(start=206388, stop=222264, step=1)	 -> DONE
Chunk 14: RangeIndex(start=222264, stop=238140, step=1)	 -> DONE
Chunk 15: RangeIndex(start=238140, stop=254016, step

Chunk 125: RangeIndex(start=1984500, stop=2000376, step=1)	 -> DONE
Chunk 126: RangeIndex(start=2000376, stop=2016252, step=1)	 -> DONE
Chunk 127: RangeIndex(start=2016252, stop=2032128, step=1)	 -> DONE
Chunk 128: RangeIndex(start=2032128, stop=2048004, step=1)	 -> DONE
Chunk 129: RangeIndex(start=2048004, stop=2063880, step=1)	 -> DONE
Chunk 130: RangeIndex(start=2063880, stop=2079756, step=1)	 -> DONE
Chunk 131: RangeIndex(start=2079756, stop=2095632, step=1)	 -> DONE
Chunk 132: RangeIndex(start=2095632, stop=2111508, step=1)	 -> DONE
Chunk 133: RangeIndex(start=2111508, stop=2127384, step=1)	 -> DONE
Chunk 134: RangeIndex(start=2127384, stop=2143260, step=1)	 -> DONE
Chunk 135: RangeIndex(start=2143260, stop=2159136, step=1)	 -> DONE
Chunk 136: RangeIndex(start=2159136, stop=2175012, step=1)	 -> DONE
Chunk 137: RangeIndex(start=2175012, stop=2190888, step=1)	 -> DONE
Chunk 138: RangeIndex(start=2190888, stop=2206764, step=1)	 -> DONE
Chunk 139: RangeIndex(start=2206764, stop=222264

Chunk 246: RangeIndex(start=3905496, stop=3921372, step=1)	 -> DONE
Chunk 247: RangeIndex(start=3921372, stop=3937248, step=1)	 -> DONE
Chunk 248: RangeIndex(start=3937248, stop=3953124, step=1)	 -> DONE
Chunk 249: RangeIndex(start=3953124, stop=3969000, step=1)	 -> DONE
Chunk 250: RangeIndex(start=3969000, stop=3984876, step=1)	 -> DONE
Chunk 251: RangeIndex(start=3984876, stop=4000752, step=1)	 -> DONE
Chunk 252: RangeIndex(start=4000752, stop=4016628, step=1)	 -> DONE
Chunk 253: RangeIndex(start=4016628, stop=4032504, step=1)	 -> DONE
Chunk 254: RangeIndex(start=4032504, stop=4048380, step=1)	 -> DONE
Chunk 255: RangeIndex(start=4048380, stop=4064256, step=1)	 -> DONE
Chunk 256: RangeIndex(start=4064256, stop=4080132, step=1)	 -> DONE
Chunk 257: RangeIndex(start=4080132, stop=4096008, step=1)	 -> DONE
Chunk 258: RangeIndex(start=4096008, stop=4111884, step=1)	 -> DONE
Chunk 259: RangeIndex(start=4111884, stop=4127760, step=1)	 -> DONE
Chunk 260: RangeIndex(start=4127760, stop=414363

Chunk 367: RangeIndex(start=5826492, stop=5842368, step=1)	 -> DONE
Chunk 368: RangeIndex(start=5842368, stop=5858244, step=1)	 -> DONE
Chunk 369: RangeIndex(start=5858244, stop=5874120, step=1)	 -> DONE
Chunk 370: RangeIndex(start=5874120, stop=5889996, step=1)	 -> DONE
Chunk 371: RangeIndex(start=5889996, stop=5905872, step=1)	 -> DONE
Chunk 372: RangeIndex(start=5905872, stop=5921748, step=1)	 -> DONE
Chunk 373: RangeIndex(start=5921748, stop=5937624, step=1)	 -> DONE
Chunk 374: RangeIndex(start=5937624, stop=5953500, step=1)	 -> DONE
Chunk 375: RangeIndex(start=5953500, stop=5969376, step=1)	 -> DONE
Chunk 376: RangeIndex(start=5969376, stop=5985252, step=1)	 -> DONE
Chunk 377: RangeIndex(start=5985252, stop=6001128, step=1)	 -> DONE
Chunk 378: RangeIndex(start=6001128, stop=6017004, step=1)	 -> DONE
Chunk 379: RangeIndex(start=6017004, stop=6032880, step=1)	 -> DONE
Chunk 380: RangeIndex(start=6032880, stop=6048756, step=1)	 -> DONE
Chunk 381: RangeIndex(start=6048756, stop=606463

Chunk 488: RangeIndex(start=7747488, stop=7763364, step=1)	 -> DONE
Chunk 489: RangeIndex(start=7763364, stop=7779240, step=1)	 -> DONE
Chunk 490: RangeIndex(start=7779240, stop=7795116, step=1)	 -> DONE
Chunk 491: RangeIndex(start=7795116, stop=7810992, step=1)	 -> DONE
Chunk 492: RangeIndex(start=7810992, stop=7826868, step=1)	 -> DONE
Chunk 493: RangeIndex(start=7826868, stop=7842744, step=1)	 -> DONE
Chunk 494: RangeIndex(start=7842744, stop=7858620, step=1)	 -> DONE
Chunk 495: RangeIndex(start=7858620, stop=7874496, step=1)	 -> DONE
Chunk 496: RangeIndex(start=7874496, stop=7890372, step=1)	 -> DONE
Chunk 497: RangeIndex(start=7890372, stop=7906248, step=1)	 -> DONE
Chunk 498: RangeIndex(start=7906248, stop=7922124, step=1)	 -> DONE
Chunk 499: RangeIndex(start=7922124, stop=7938000, step=1)	 -> DONE
Chunk 500: RangeIndex(start=7938000, stop=7953876, step=1)	 -> DONE
Chunk 501: RangeIndex(start=7953876, stop=7969752, step=1)	 -> DONE
Chunk 502: RangeIndex(start=7969752, stop=798562

Chunk 609: RangeIndex(start=9668484, stop=9684360, step=1)	 -> DONE
Chunk 610: RangeIndex(start=9684360, stop=9700236, step=1)	 -> DONE
Chunk 611: RangeIndex(start=9700236, stop=9716112, step=1)	 -> DONE
Chunk 612: RangeIndex(start=9716112, stop=9731988, step=1)	 -> DONE
Chunk 613: RangeIndex(start=9731988, stop=9747864, step=1)	 -> DONE
Chunk 614: RangeIndex(start=9747864, stop=9763740, step=1)	 -> DONE
Chunk 615: RangeIndex(start=9763740, stop=9779616, step=1)	 -> DONE
Chunk 616: RangeIndex(start=9779616, stop=9795492, step=1)	 -> DONE
Chunk 617: RangeIndex(start=9795492, stop=9811368, step=1)	 -> DONE
Chunk 618: RangeIndex(start=9811368, stop=9827244, step=1)	 -> DONE
Chunk 619: RangeIndex(start=9827244, stop=9843120, step=1)	 -> DONE
Chunk 620: RangeIndex(start=9843120, stop=9858996, step=1)	 -> DONE
Chunk 621: RangeIndex(start=9858996, stop=9874872, step=1)	 -> DONE
Chunk 622: RangeIndex(start=9874872, stop=9890748, step=1)	 -> DONE
Chunk 623: RangeIndex(start=9890748, stop=990662

Chunk 727: RangeIndex(start=11541852, stop=11557728, step=1)	 -> DONE
Chunk 728: RangeIndex(start=11557728, stop=11573604, step=1)	 -> DONE
Chunk 729: RangeIndex(start=11573604, stop=11589480, step=1)	 -> DONE
Chunk 730: RangeIndex(start=11589480, stop=11605356, step=1)	 -> DONE
Chunk 731: RangeIndex(start=11605356, stop=11621232, step=1)	 -> DONE
Chunk 732: RangeIndex(start=11621232, stop=11637108, step=1)	 -> DONE
Chunk 733: RangeIndex(start=11637108, stop=11652984, step=1)	 -> DONE
Chunk 734: RangeIndex(start=11652984, stop=11668860, step=1)	 -> DONE
Chunk 735: RangeIndex(start=11668860, stop=11684736, step=1)	 -> DONE
Chunk 736: RangeIndex(start=11684736, stop=11700612, step=1)	 -> DONE
Chunk 737: RangeIndex(start=11700612, stop=11716488, step=1)	 -> DONE
Chunk 738: RangeIndex(start=11716488, stop=11732364, step=1)	 -> DONE
Chunk 739: RangeIndex(start=11732364, stop=11748240, step=1)	 -> DONE
Chunk 740: RangeIndex(start=11748240, stop=11764116, step=1)	 -> DONE
Chunk 741: RangeInde

Chunk 845: RangeIndex(start=13415220, stop=13431096, step=1)	 -> DONE
Chunk 846: RangeIndex(start=13431096, stop=13446972, step=1)	 -> DONE
Chunk 847: RangeIndex(start=13446972, stop=13462848, step=1)	 -> DONE
Chunk 848: RangeIndex(start=13462848, stop=13478724, step=1)	 -> DONE
Chunk 849: RangeIndex(start=13478724, stop=13494600, step=1)	 -> DONE
Chunk 850: RangeIndex(start=13494600, stop=13510476, step=1)	 -> DONE
Chunk 851: RangeIndex(start=13510476, stop=13526352, step=1)	 -> DONE
Chunk 852: RangeIndex(start=13526352, stop=13542228, step=1)	 -> DONE
Chunk 853: RangeIndex(start=13542228, stop=13558104, step=1)	 -> DONE
Chunk 854: RangeIndex(start=13558104, stop=13573980, step=1)	 -> DONE
Chunk 855: RangeIndex(start=13573980, stop=13589856, step=1)	 -> DONE
Chunk 856: RangeIndex(start=13589856, stop=13605732, step=1)	 -> DONE
Chunk 857: RangeIndex(start=13605732, stop=13621608, step=1)	 -> DONE
Chunk 858: RangeIndex(start=13621608, stop=13637484, step=1)	 -> DONE
Chunk 859: RangeInde

Chunk 963: RangeIndex(start=15288588, stop=15304464, step=1)	 -> DONE
Chunk 964: RangeIndex(start=15304464, stop=15320340, step=1)	 -> DONE
Chunk 965: RangeIndex(start=15320340, stop=15336216, step=1)	 -> DONE
Chunk 966: RangeIndex(start=15336216, stop=15352092, step=1)	 -> DONE
Chunk 967: RangeIndex(start=15352092, stop=15367968, step=1)	 -> DONE
Chunk 968: RangeIndex(start=15367968, stop=15383844, step=1)	 -> DONE
Chunk 969: RangeIndex(start=15383844, stop=15399720, step=1)	 -> DONE
Chunk 970: RangeIndex(start=15399720, stop=15415596, step=1)	 -> DONE
Chunk 971: RangeIndex(start=15415596, stop=15431472, step=1)	 -> DONE
Chunk 972: RangeIndex(start=15431472, stop=15447348, step=1)	 -> DONE
Chunk 973: RangeIndex(start=15447348, stop=15463224, step=1)	 -> DONE
Chunk 974: RangeIndex(start=15463224, stop=15479100, step=1)	 -> DONE
Chunk 975: RangeIndex(start=15479100, stop=15494976, step=1)	 -> DONE
Chunk 976: RangeIndex(start=15494976, stop=15510852, step=1)	 -> DONE
Chunk 977: RangeInde

In [25]:
df_with_label = pd.concat(df_chunks, ignore_index=True).sort_values(['bookingID', 'second']).reset_index(drop=True)

In [26]:
print(df_with_label.shape)
df_with_label.head()

(15876650, 12)


,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed,label
0,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991,0
1,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,0.228454,0
2,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454,0
3,0,8.0,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454,0
4,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,0.228454,0


In [19]:
df_with_label.to_csv(r'safety/data_cleaned_combined.csv', index=False)